In [ ]:
import pandas as pd
import numpy as np


In [22]:
df = pd.read_csv('/content/Active_Business_License_Tax_Certificate_20260217.csv')
df.head()

# # Standardize
df['City'] = df['City'].str.strip().str.upper()
df['State'] = df['State'].str.strip().str.upper()

# Filter Seattle, WA
df_seattle = df[(df['City'] == 'SEATTLE') & (df['State'] == 'WA')].copy()

# Coffee NAICS codes:
# 722515 = Snack and Nonalcoholic Beverage Bars (Coffee shops)
# 722513 = Limited-Service Restaurants (optional inclusion)
df_seattle.dtypes



,0
Business Legal Name,object
Trade Name,object
Ownership Type,object
NAICS Code,int64
NAICS Description,object
License Start Date,int64
Street Address,object
City,object
State,object
Zip,object


In [23]:
coffee_naics = [722515, 722513]

df_coffee = df_seattle[df_seattle['NAICS Code'].isin(coffee_naics)].copy()


In [24]:
df_seattle.head()

,Business Legal Name,Trade Name,Ownership Type,NAICS Code,NAICS Description,License Start Date,Street Address,City,State,Zip,Business Phone,City Account Number,UBI
0,ABDURAKHMANOV RASULZHON,RASULZHON ABDURAKHMANOV,Sole proprietorship,485320,Limousine Service,20140814,706 UNION ST # 409,SEATTLE,WA,98101,2062579788,7748490686809,NaN
2,BUI PHUONG,BUI PHUONG,Sole proprietorship,721191,Bed-and-Breakfast Inns,20190827,2324 1ST AVE #313,SEATTLE,WA,98121,4255770134,8410320755821,6.044996e+15
3,CITY TAXI LLC,SILVER CAB LLC DBA SILVER CAB # 2169,LLC - Multi Member,485310,Taxi and Ridesharing Services,20240510,9417 27TH AVE SW # F,SEATTLE,WA,98126-3998,2068220220,8760300792736,6.054315e+15
4,CRUZ BRENDAN J,PHENOMENAL PHILLIE,Sole proprietorship,311811,Retail Bakeries,20220801,4130 PALATINE AVE N,SEATTLE,WA,98103,2153916626,8621920778133,6.049137e+08
5,DAVID ANTHONY LEWIS PS,DAVID ANTHONY LEWIS PS,Corporation,611699,All Other Miscellaneous Schools and Instruction,20220203,8707 16TH AVE SW,SEATTLE,WA,98106-2377,2065519336,8590290774829,6.035059e+15


In [25]:
df_coffee.head()

,Business Legal Name,Trade Name,Ownership Type,NAICS Code,NAICS Description,License Start Date,Street Address,City,State,Zip,Business Phone,City Account Number,UBI
108,12TH AVE SLICE LLC,12TH AVE SLICE,LLC - Multi Member,722513,Limited-Service Restaurants,20240201,1542 12TH AVE,SEATTLE,WA,98122-3908,4258693670,8731030789643,6.053495e+15
116,1313 PIZZA LLC,MAGGIE POINT,LLC - Multi Member,722513,Limited-Service Restaurants,20230508,4736 UNIVERSITY WAY NE,SEATTLE,WA,98105,2066973120,8744920791084,6.049895e+08
229,203 DEGREES FAHRENHEIT CORP,203 DEGREES FAHRENHEIT,Corporation,722515,Snack and Nonalcoholic Beverage Bars,20210301,610 TERRY AVE N # 170,SEATTLE,WA,98109,6088433502,8876580804992,6.041976e+15
249,21 MERCER ST INC,TS MCHUGHS,Corporation,722513,Limited-Service Restaurants,19900630,21 MERCER ST,SEATTLE,WA,98109,2062821910,1211000122387,6.012356e+15
347,2PIZANOS LLC,PAULS ITALIAN KITCHEN,LLC - Multi Member,722513,Limited-Service Restaurants,20170301,4333 UNIVERSITY WAY NE,SEATTLE,WA,98105-5808,2065254493,8044930717629,6.040802e+15


In [26]:
df_coffee.shape

(1601, 13)

In [28]:
df_coffee['Ownership Type'].value_counts()


,count
Ownership Type,
Corporation,600
LLC - Multi Member,529
LLC - Single Member,242
Sole proprietorship,103
Other,65
Limited Liability Limited Ptrs,18
General Partnership,17
Association,14
Limited Liability Ptrs LLP,8


In [33]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 5.1 MB/s eta 0:00:00


In [35]:
import osmnx as ox
import pandas as pd

place = "Seattle, Washington, USA"

tags = {"amenity": "cafe"}

gdf = ox.features_from_place(place, tags)

# Keep useful columns if they exist
cols = [c for c in ["name", "amenity", "addr:street", "addr:housenumber", "geometry"] if c in gdf.columns]
gdf = gdf[cols]

gdf = gdf[gdf["name"].notna()]

print(gdf.head())
print(len(gdf))


                              name amenity                  addr:street  \
element id                                                                
node    440839204      Lula Coffee    cafe  California Avenue Southwest   
        440843430  Uptown Espresso    cafe     Southwest Edmunds Street   
        441510758  Alki Coffee Co.    cafe        Alki Avenue Southwest   
        457772972        Starbucks    cafe           West McGraw Street   
        457773030  Uptown Espresso    cafe           West McGraw Street   

                  addr:housenumber                     geometry  
element id                                                       
node    440839204             4453  POINT (-122.38699 47.56322)  
        440843430             4301  POINT (-122.38756 47.55917)  
        441510758             2742   POINT (-122.4112 47.57882)  
        457772972             3300  POINT (-122.39981 47.63971)  
        457773030             3223  POINT (-122.39946 47.63942)  
749


In [37]:
import folium

# Convert non-point geometries to centroids
gdf["geometry"] = gdf["geometry"].centroid

# Now safe to access x and y
center_lat = gdf.geometry.y.mean()
center_lon = gdf.geometry.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

m


/tmp/ipython-input-2869856594.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["geometry"] = gdf["geometry"].centroid
